# ApiLinker Research Tutorial

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/kkartas/APILinker/HEAD?labpath=examples%2FApiLinker_Research_Tutorial.ipynb)

**🚀 Try this notebook in your browser!** Click the Binder badge above to launch an interactive version of this tutorial (no installation required).

---

This notebook demonstrates how to use ApiLinker for research workflows, including literature searches, data integration, and cross-platform research.

## Table of Contents
1. [Installation and Setup](#installation)
2. [Basic API Integration](#basic-integration)
3. [Research Connectors](#research-connectors)
4. [Literature Search Workflow](#literature-search)
5. [Data Transformation and Mapping](#data-transformation)
6. [Cross-Platform Research](#cross-platform)
7. [Visualization Examples](#visualization)


## Installation and Setup {#installation}

First, let's install ApiLinker and import the necessary modules.

**Note for Binder users**: ApiLinker should be automatically installed from the repository. If you encounter import errors, the installation may have failed. Check the Binder build logs or try restarting the kernel.


In [ ]:
# Note: If running on Binder, ApiLinker is already installed!
# If running locally, uncomment the line below:
# !pip install apilinker

# Diagnostic information
import sys
import os
print("Python version:", sys.version)
print("Python path:", sys.executable)
print()

# Import ApiLinker (always available)
from apilinker import ApiLinker
import apilinker

print(f"ApiLinker version: {apilinker.__version__}")
print(f"ApiLinker location: {apilinker.__file__}")
print()

# Check if connectors directory exists
connectors_path = os.path.join(os.path.dirname(apilinker.__file__), 'connectors')
if os.path.exists(connectors_path):
    print(f"✅ Connectors directory found: {connectors_path}")
    print(f"   Contents: {os.listdir(connectors_path)}")
else:
    print(f"⚠️  Connectors directory not found at: {connectors_path}")
print()

# Try to import research connectors (may not be available in older versions)
try:
    from apilinker import (
        NCBIConnector, ArXivConnector, CrossRefConnector,
        SemanticScholarConnector, PubChemConnector, ORCIDConnector,
        GitHubConnector, NASAConnector
    )
    RESEARCH_CONNECTORS_AVAILABLE = True
    print("✅ ApiLinker and all research connectors imported successfully!")
except ImportError as e:
    RESEARCH_CONNECTORS_AVAILABLE = False
    print(f"⚠️  ApiLinker imported, but research connectors are not available.")
    print(f"   Error: {e}")
    print()
    print("   Debugging information:")
    # Try direct import to see the actual error
    try:
        from apilinker.connectors.scientific.ncbi import NCBIConnector
        print("   ✅ Direct import from connectors.scientific.ncbi works!")
        print("   ⚠️  Issue is likely in __init__.py import handling")
    except Exception as direct_error:
        print(f"   ❌ Direct import also failed: {direct_error}")
    print()
    print("   This may happen if:")
    print("   - Using an older version from PyPI")
    print("   - The repository installation didn't include connectors")
    print("   - There's an import error in the connector modules")
    print()
    print("   On Binder, check the build logs to see if installation succeeded.")
    print("   You can still use ApiLinker for general API integration!")

import pandas as pd
import json
from datetime import datetime


## Basic API Integration {#basic-integration}

Let's start with a simple example of connecting two APIs and mapping data between them.


In [ ]:
# Initialize ApiLinker
linker = ApiLinker()

# Example: Connect to a public API (no authentication required)
linker.add_source(
    type="rest",
    base_url="https://api.github.com",
    endpoints={
        "get_repo": {
            "path": "/repos/kkartas/APILinker",
            "method": "GET"
        }
    }
)

# Fetch data
repo_data = linker.fetch("get_repo")

# Display results
print(f"Repository: {repo_data.get('name')}")
print(f"Description: {repo_data.get('description')}")
print(f"Stars: {repo_data.get('stargazers_count')}")
print(f"Language: {repo_data.get('language')}")


## Research Connectors {#research-connectors}

ApiLinker includes 8 specialized research connectors. Let's explore them:


In [ ]:
# Initialize research connectors
# Note: Some connectors require API keys or email addresses

if not RESEARCH_CONNECTORS_AVAILABLE:
    print("❌ Research connectors are not available. Please ensure you're using the latest version.")
    print("   On Binder, this should work automatically. If not, the repository may need to be updated.")
else:
    # Scientific Literature
    ncbi = NCBIConnector(email="researcher@university.edu")  # Replace with your email
    arxiv = ArXivConnector()  # No API key required
    crossref = CrossRefConnector(email="researcher@university.edu")
    semantic = SemanticScholarConnector()  # Optional API key for higher rate limits
    
    # Chemical & Biological Data
    pubchem = PubChemConnector()  # No API key required
    orcid = ORCIDConnector()  # Public API, no key required
    
    # Code & Data
    github = GitHubConnector()  # Optional token for higher rate limits
    nasa = NASAConnector()  # Uses DEMO_KEY by default (limited rate, get your key from api.nasa.gov)
    
    print("✅ All research connectors initialized!")

## Literature Search Workflow {#literature-search}

Let's create a comprehensive literature search across multiple databases:


In [ ]:
# Initialize variables (needed for visualization even if connectors aren't available)
topic = "machine learning protein folding"
pubmed_ids = []
arxiv_results = []
semantic_papers = []

if not RESEARCH_CONNECTORS_AVAILABLE:
    print("⚠️  Research connectors are not available. Skipping this section.")
    print("   Please check the installation or use the basic API integration examples below.")
    print(f"   (Topic would have been: {topic})")
else:
    print(f"🔍 Searching for: {topic}")
    print("=" * 60)
    
    # Search PubMed (biomedical literature)
    print("\n📚 Searching PubMed...")
    try:
        pubmed_results = ncbi.search_pubmed(topic, max_results=10)
        pubmed_ids = pubmed_results.get('esearchresult', {}).get('idlist', [])
        print(f"   Found {len(pubmed_ids)} papers")
    except Exception as e:
        print(f"   Error: {e}")
        pubmed_ids = []
    
    # Search arXiv (preprints)
    print("\n📄 Searching arXiv...")
    try:
        arxiv_results = arxiv.search_papers(topic, max_results=10)
        print(f"   Found {len(arxiv_results)} papers")
    except Exception as e:
        print(f"   Error: {e}")
        arxiv_results = []
    
    # Search Semantic Scholar
    print("\n🤖 Searching Semantic Scholar...")
    try:
        semantic_results = semantic.search_papers(topic, max_results=10)
        semantic_papers = semantic_results.get('data', [])
        print(f"   Found {len(semantic_papers)} papers")
    except Exception as e:
        print(f"   Error: {e}")
        semantic_papers = []
    
    print("\n✅ Literature search complete!")


### Visualize Search Results


In [ ]:
import matplotlib.pyplot as plt

# Create a summary of results
results_summary = {
    'Database': ['PubMed', 'arXiv', 'Semantic Scholar'],
    'Papers Found': [len(pubmed_ids), len(arxiv_results), len(semantic_papers)]
}

df = pd.DataFrame(results_summary)
print("\n📊 Search Results Summary:")
print(df.to_string(index=False))

# Create a simple bar chart
if len(df) > 0 and (len(pubmed_ids) > 0 or len(arxiv_results) > 0 or len(semantic_papers) > 0):
    plt.figure(figsize=(8, 5))
    plt.bar(df['Database'], df['Papers Found'], color=['#2E86AB', '#A23B72', '#F18F01'])
    plt.title(f'Literature Search Results: {topic}')
    plt.ylabel('Number of Papers')
    plt.xlabel('Database')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
elif not RESEARCH_CONNECTORS_AVAILABLE:
    print("\n⚠️  Visualization skipped: Research connectors are not available.")
    print("   Install ApiLinker from the repository to see search results visualization.")
else:
    print("\n⚠️  No results to visualize. Try running the search cells above first.")


## Next Steps

1. **Explore the documentation**: Check out [docs/research_workflows.md](../docs/research_workflows.md)
2. **Try more examples**: See [examples/comprehensive_research_examples.py](../examples/comprehensive_research_examples.py)
3. **Configure your APIs**: Set up API keys for higher rate limits
4. **Create your own workflows**: Use ApiLinker to automate your research data collection

## Resources

- **GitHub**: https://github.com/kkartas/APILinker
- **Documentation**: https://apilinker.readthedocs.io/
- **PyPI**: https://pypi.org/project/apilinker/

---

*This notebook demonstrates ApiLinker's capabilities for research workflows. Modify the examples to fit your research needs!*
